# Ghouls, Goblins, and Ghosts... Boo!

This is my first Kaggle competition and first try in machine learning. 

Link: https://www.kaggle.com/c/ghouls-goblins-and-ghosts-boo

Problem description: create classification algorithm to distinguish type of monsters by given features.

## 1 - Packages

In [ ]:
from sklearn import preprocessing, ensemble, model_selection

import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns

In [ ]:
%pylab inline

## 2 - Overview of the Dataset

Loading data:

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_csv('../input/ghouls-goblins-and-ghosts-boo/train.csv.zip', header=0, sep=',')
test_data = pd.read_csv('../input/ghouls-goblins-and-ghosts-boo/test.csv.zip', header=0, sep=',')

Shape of datasets:

In [ ]:
print('Shape of train data: ', train_data.shape)
print('Shape of test data: ', test_data.shape)

First 5 rows of datasets:

In [ ]:
train_data.head()

In [ ]:
test_data.head()

Check column names:

In [ ]:
print('Column names:', list(train_data.columns))

Remove id columns from datasets as it won't be used:

In [ ]:
train_data = train_data.drop(['id'], axis=1)

test_id = test_data['id']
test_data = test_data.drop(['id'], axis=1)

Visualising train dataset as a pairplot. Represents float values of columns 'bone_length', 'rotting_flesh', 'hair_length', 'has_soul'.

In [ ]:
sns.pairplot(train_data.drop('color', axis = 1), hue = 'type', palette = 'muted', diag_kind='kde')

Dataset contains categorical value column 'color'. Let's visualise it.

In [ ]:
sns.countplot(x='color', hue='type', data=train_data)

Compare general amount of samples to classify for each value.

In [ ]:
sns.countplot(x='type', data=train_data)

Next we are splitting data to target and features sets. Let's apply label encoding for target values(replace string values by int) in target set and for 'color' column in feature set.

In [ ]:
color_le = preprocessing.LabelEncoder()

train_data_x = train_data
color_le.fit(train_data_x['color'])
train_data_x['color_int'] = color_le.transform(train_data_x['color'])
train_data_x = train_data_x.drop(['color', 'type'], axis=1)
train_data_x = train_data_x.to_numpy()

type_le = preprocessing.LabelEncoder()

train_data_y = train_data['type']
type_le.fit(train_data_y)
train_data_y = type_le.transform(train_data_y)

print('Unique type values:', train_data.type.unique())
print()
print('Original train_set_y:', np.array(train_data.type[:5]))
print('Encoded train_set_y:', train_data_y[:5])

Preprocessing test data for final prediction is the same as for train_data_x.

In [ ]:
test_data = test_data
color_le.fit(test_data['color'])
test_data['color_int'] = color_le.transform(test_data['color'])
test_data = test_data.drop(['color'], axis=1)
test_data = test_data.to_numpy()

Preprocessed test data looks like:

In [ ]:
print('test_data:')
print(test_data[:5])

Final train x and y sets look like:

In [ ]:
print('train_data_x:')
print(train_data_x[:5])
print('shape:', train_data_x.shape, 'type:', type(train_data_x))
print()
print('train_data_y:')
print(train_data_y[:5])
print('shape:', train_data_y.shape, 'type:', type(train_data_y))

## 3 - Model

In this project I am going to use Random Forest Classifier to classify types of creatures. We need to determine hyperparameter - number of trees in forest. Let's look through range of numbers 1, 5, ... 50. We will use cross validation with 3 foldes to find the best parameter.

In [ ]:
n_trees = [1] + list(range(5, 55, 5))
scoring = []
for n_tree in n_trees:
    estimator = ensemble.RandomForestClassifier(n_estimators = n_tree, min_samples_split=5, random_state=1)
    score = model_selection.cross_val_score(estimator, train_data_x, train_data_y, 
                                             scoring = 'accuracy', cv = 3)    
    scoring.append(score)
scoring = np.asmatrix(scoring)

scoring

In [ ]:
pylab.plot(n_trees, scoring.mean(axis = 1), marker='.', label='RandomForest')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right')

Alternate way to find best value of hyperparameter is to use GridSearchCV:

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators': n_trees}

grid_search = GridSearchCV(ensemble.RandomForestClassifier(min_samples_split=5, random_state=1), params, cv=3, scoring='accuracy')

grid_search.fit(train_data_x, train_data_y)

print('Best n of trees: {}, best accuracy score: {}'.format(grid_search.best_params_['n_estimators'], grid_search.best_score_))

It's clearly seen, that both searches have the same results. So, we will use 20 trees for Random Forest Classifier. Now we will divide train data into train and test sets for our model.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_set_x, test_set_x, train_set_y, test_set_y = train_test_split(train_data_x, train_data_y, test_size=0.33, random_state=42)

Now let's fit our model and predict y for test set. Then calculate accuracy score for test set.

In [ ]:
best_n_tree = grid_search.best_params_['n_estimators']
model = ensemble.RandomForestClassifier(n_estimators = n_tree, min_samples_split=5, random_state=1)
model.fit(train_set_x, train_set_y)
pred_y = model.predict(test_set_x)

score = accuracy_score(test_set_y, pred_y)
print('Accuracy score:', score)

Finally, let's predict the 'type' variable for test_data.

In [ ]:
test_pred = model.predict(test_data)
encoded_test_pred = type_le.inverse_transform(test_pred)
print('Test predictions:', encoded_test_pred[:5])

In [ ]:
submission = pd.DataFrame({'id': test_id, 'type':encoded_test_pred})
submission.to_csv('submission.csv', index=False)